In [111]:
import numpy as np
import pandas as pd
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
import re
from collections import Counter, defaultdict
from nltk.corpus import stopwords

In [112]:
df = pd.read_csv("D:\\Courses\\Sem 8 2021-22\\COL865\\Project Dataset\\CodaLab\\Constraint_English_Train - Sheet1.csv")

In [134]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [151]:
def clean_text(
    string: str, 
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=[]):
    

    string = re.sub(r'https?://\S+|www\.\S+', '', string)


    string = re.sub(r'<.*?>', '', string)


    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, " ") 

    string = string.lower()


    string = ' '.join([word for word in string.split() if word not in stop_words])

    string = re.sub(r'\s+', ' ', string).strip()

    return string   

In [152]:
def normalize(X):
    mean = np.mean(X)
    stddev = np.std(X)
    return (X - mean) * (1 / stddev)

In [155]:
vectorizer = TfidfVectorizer()
vectorizer = CountVectorizer()
def dataset(pathname, model = 'TF_IDF', default = 'test', vectorizer = vectorizer):
    
    df = pd.read_csv(pathname)
    
    if(model == 'TF_IDF' or model == 'Doc_term'):
        corpus = []
        for x in df['tweet']:
            corpus.append(clean_text(remove_emojis(x)))
        
        if(default == 'train'):
            X = vectorizer.fit_transform(corpus)
        elif(default == 'test'):
            X = vectorizer.transform(corpus)
        
        print(X.shape)
        
        
        Y = [1 if y == 'real' else 0 for y in df['label']]

        return X, Y
    elif(model == 'Doc2Vec'):
        print(1)
        

In [156]:
X_train, Y_train = dataset("D:\\Courses\\Sem 8 2021-22\\COL865\\Project Dataset\\CodaLab\\Constraint_English_Train - Sheet1.csv", default = 'train')

(6420, 14278)


In [100]:
from sklearn.linear_model import LogisticRegression

In [101]:
clf = LogisticRegression(random_state=0, solver = 'liblinear').fit(X_train, Y_train)

In [102]:
clf.predict(X_train[3, :])

array([1])

In [103]:
y_pred = [int(clf.predict(X_train[i, :])) for i in range(6420)]

In [104]:
y_pred

[1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,


In [105]:
def accuracy(Y, y):
    acc = 0
    for i in range(len(Y)):
        if Y[i] == y[i]:
            acc +=1
    return acc/len(Y)
            

In [106]:
accuracy(Y_train, y_pred)

0.9538940809968848

In [148]:
X_test, y_test =  dataset("D:\\Courses\\Sem 8 2021-22\\COL865\\Project Dataset\\CodaLab\\english_test_with_labels - Sheet1.csv", default = 'test')

(2140, 14278)


In [149]:
ytest_pred = [int(clf.predict(X_train[i, :])) for i in range(2140)]

In [150]:
accuracy(y_test, ytest_pred)

0.5275700934579439